In [ ]:
import pandas as pd
import re

In [ ]:
#!pip install google_drive_downloader
#import google_drive_downloader
#from google_drive_downloader import GoogleDriveDownloader as gdd
import shutil

def get_tweets_pos():
    gdd.download_file_from_google_drive(
        file_id = '1VD86hdSUECFn5fSunBJv9FR2NZGA6PfV',
        dest_path = './tweets_pos_clean.txt',
    )

def get_tweets_neg():
    gdd.download_file_from_google_drive(
        file_id = '1FfZx6qZkkMLXU1hJGjH8dFjiIfiyebVx',
        dest_path = './tweets_neg_clean.txt',
    )  

get_tweets_pos()
get_tweets_neg()

In [ ]:
df_pos= pd.read_csv('../input/tweets-benaice/tweets_pos_clean.txt', delimiter = "\t", names=['Text'])
df_pos['label']=1 # Añado una columna de 1 --> POSITIVO
df_neg = pd.read_csv('../input/tweets-benaice/tweets_neg_clean.txt', delimiter = "\t", names=['Text'])
df_neg['label']=0 # Añado una columna de 0 --> NEGATIVO

In [ ]:
df = pd.concat([df_pos, df_neg]) # Concateno los dataframes
df = df.sample(frac=1).reset_index(drop=True) # Los mezclo y reemplazo los indices
df

In [ ]:
df.to_csv('tweets_clasificados.csv', index = False) # Lo guardo en formato CSV 

In [ ]:
df = pd.read_csv('tweets_clasificados.csv')
df.head()

In [ ]:
df['label'].value_counts()

In [ ]:
df.shape

In [ ]:
import transformers

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
column_names = list(df.columns)
print(column_names)

In [ ]:
data = load_dataset('csv',data_files ='tweets_clasificados.csv',column_names = column_names, skiprows = 1)
data_train = data['train']

In [ ]:
data

In [ ]:
data_train

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

In [ ]:
def clean_text(example):

    def cleaning_function(text):
        text_processed = text
        text_processed = re.sub('https:\/\/t\.co\/[a-zA-Z0-9]+','enlace',text_processed)
        text_processed = re.sub('@(\w){1,15}','usuario', text_processed)
        text_processed = re.sub('#[a-zA-Z0-9]+','tema ', text_processed)
        text_processed = re.sub(r"http\S+",'',text_processed) #Removing hyperlinks
        text_processed = re.sub(r'RT[\s]+','',text_processed) #Removing the 'RT' word
        text_processed = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)', 'MENCIÓN',text_processed) # Removing @mentions
        text_processed = re.sub(r'#','',text_processed) # Removing the '#' symbol
        text_processed = re.sub(r':\)','',text_processed) # Removing the ':)'
        text_processed = re.sub(r':-\)','',text_processed) # Removing the ':-)'
        text_processed = re.sub(r':\(','',text_processed) # Removing the ':('
        text_processed = re.sub(r':-\(','',text_processed) # Removing the ':-('
        text_processed = re.sub(r':','',text_processed) # Removing the ':' symbol
        pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        text_processed = pattern.sub(r'EMOJI',text_processed)
        return text_processed
    texts_cleaned = []
    for text in example['Text']:
        texts_cleaned.append(cleaning_function(text))
    return {'Text':texts_cleaned,'label':example['label']}


data_train = data_train.map(clean_text, batched=True)
data = data.map(clean_text, batched=True)

In [ ]:
data_train[:3]
data['train'][:3]

In [ ]:
from datasets import Dataset
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)

subsample = 10000 #sustituir por -1 para usar todo el dataset
data_train = Dataset.from_dict(data_train[:subsample])
tokenized_datasets = data_train.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
data_split = tokenized_datasets.train_test_split(test_size=0.2, shuffle= True)

X_train = data_split['train']
X_test = data_split['test']

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import torch 

#model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es", num_labels=2)

training_args = TrainingArguments("test_trainer",evaluation_strategy="epoch",num_train_epochs=5, per_device_train_batch_size=6, learning_rate = 7e-07, save_total_limit = 1)

In [ ]:
X_train['label'][:15]

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=X_train, eval_dataset=X_test)

trainer.train()

In [ ]:
import numpy as np
from datasets import load_metric 

metric_accuracy = load_metric("accuracy")
metric_precision = load_metric("precision")
metric_recall = load_metric("recall")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric_accuracy.compute(predictions=predictions, references=labels)
    precision = metric_precision.compute(predictions=predictions, references=labels)
    recall = metric_recall.compute(predictions=predictions, references=labels)
    return accuracy#, precision, recall


In [ ]:
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_test,
    compute_metrics=compute_metrics,
)
history = trainer.evaluate()

In [ ]:
print(history)

In [ ]:
model.save_pretrained('distill_bnaice_model')

La versión 6 funciona muy bien (está descargada)